Explore the graph_tool package, in order to do a compare & contrast of SBMs and Infomap

In [65]:
import graph_tool
import graph_tool.inference
import numpy as np
import matplotlib.pyplot as plt
from palettable.colorbrewer.qualitative import Set3_12
%matplotlib inline

In [91]:
edgelistArg= "3p8t.4.0.dat"
g = graph_tool.load_graph_from_csv(edgelistArg, directed=False,
                                   skip_first=True, csv_options={"delimiter": " "})
print(g)

In [92]:
state = graph_tool.inference.minimize_nested_blockmodel_dl(g, deg_corr=True)

In [93]:
hierarchical_partition = []
for i, innerstate in enumerate(state.get_levels()):
    hierarchical_partition.append(list(state.project_level(i).b.get_array()))
print(hierarchical_partition)

In [94]:
state.draw()

In [68]:
def plotStripeDiagram(data):
    """
    Plot the partition as a set of "stripe" plots.
    """
    # Convert the list (or nested list) to a numpy array, for use with imshow.
    stripes = np.asarray(data, dtype=int)
    numPlots =  np.shape(stripes)[0]

    fig, axes = plt.subplots(nrows=numPlots, figsize=(5, 5), sharex=True)

    for i, ax in enumerate(axes):
        ax.imshow(
            np.vstack(
                2 * [stripes[i, :]]),  # vstack otherwise imshow complains
            aspect=100,
            cmap=Set3_12.mpl_colormap,
            interpolation="nearest")
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_visible(False)

#     plt.suptitle(self.pdbref)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.xlabel('Residue number')
    plt.show()

In [69]:
plotStripeDiagram(hierarchical_partition)

The above shows the block model hierarchy. Need to reverse the order and trim the "all-one-community" level.

In [95]:
hierarchical_partition =hierarchical_partition[::-1][1:]

In [73]:
plotStripeDiagram(hierarchical_partition)

In [76]:
print(len(hierarchical_partition[-1]))
print(len(set(hierarchical_partition[-1])))

In [78]:
state = state.copy(sampling=True)

In [79]:
graph_tool.inference.mcmc_equilibrate(state)

In [80]:
state.draw()

In [81]:
hierarchical_partition_new = []
for i, innerstate in enumerate(state.get_levels()):
    hierarchical_partition_new.append(list(state.project_level(i).b.get_array()))
hierarchical_partition_new = hierarchical_partition_new[::-1][1:]

In [82]:
plotStripeDiagram(hierarchical_partition_new)

So it's looking like mcmc_equilibriate, as I'm currently doing it, isn't the one. 

Compare with Infomap:

In [97]:
%load_ext autoreload
%autoreload 2

import proteinnetworks

In [84]:
db = proteinnetworks.database.Database()

In [111]:
inputArgs = {"scaling": 4.5,
"edgelisttype": "residue",
"hydrogenstatus": "noH",
"pdbref": "3p8t",
"database": db}
network = proteinnetworks.network.Network(**inputArgs)
from bson.objectid import ObjectId
partitionArgs = {"pdbref": "3p8t",
                 "edgelistid": ObjectId(network.edgelistid),
                 "detectionmethod": "Infomap",
                "N": 100,
                "database": db}
infomap_partition = proteinnetworks.partition.Partition(**partitionArgs)
infomap_partition.plotStripeDiagram(includePFAMDomains=True)

In [96]:
plotStripeDiagram(hierarchical_partition)

In [110]:
proteinnetworks.insight.getNMI(
    [x+1 for x in hierarchical_partition[0]], infomap_partition.data[0])

### Compare the SBM and Infomap outputs visually using Pymol

In [64]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import proteinnetworks
import glob
import matplotlib.pyplot as plt
import numpy as np
from palettable.colorbrewer.qualitative import Set3_12
import os
import subprocess

In [2]:
db = proteinnetworks.database.Database()

In [7]:
from bson.objectid import ObjectId

In [39]:
def plotStripeDiagramSideBySide(infomapData, SBMdata, title):
    """Plot the partition as a set of "stripe" plots."""
    # Convert the list (or nested list) to a numpy array, for use with imshow.
    # TODO need a check for if either array is 1d
    stripes1 = np.atleast_2d(np.asarray(infomapData, dtype=int))
    stripes2 = np.atleast_2d(np.asarray(SBMdata, dtype=int))
    numInfomapArrays = np.shape(stripes1)[0]
    numSBMArrays = np.shape(stripes2)[0]
    print(np.shape(stripes1), np.shape(stripes2))
    while numInfomapArrays < numSBMArrays:

        # Pad with ones until both arrays are the same shape
        stripes1 = np.vstack([stripes1, [1] * np.shape(stripes1)[1]])
        numInfomapArrays = np.shape(stripes1)[0]

    while numSBMArrays < numInfomapArrays:
        # Pad with ones until both arrays are the same shape
        stripes2 = np.vstack([stripes2, [1] * np.shape(stripes2)[1]])
        numSBMArrays = np.shape(stripes2)[0]

    assert numInfomapArrays == numSBMArrays
    numPlots = numInfomapArrays

    fig, axes = plt.subplots(
        nrows=numPlots, ncols=2, figsize=(8, 8), sharex=True)
    try:
        stripes3 = np.stack([stripes1, stripes2], axis=2)
    except ValueError:
        print(np.shape(stripes1), np.shape(stripes2))
    assert np.all(stripes3[:, :, 0] == stripes1)
    for i in range(numInfomapArrays):
        axes[i, 0].imshow(
            np.vstack(
                2 * [stripes3[i, :, 0]]),  # vstack otherwise imshow complains
            aspect=100,
            cmap=Set3_12.mpl_colormap,
            interpolation="nearest")
        axes[i, 0].xaxis.set_ticks_position('bottom')
        axes[i, 0].yaxis.set_visible(False)
    for i in range(numSBMArrays):
        axes[i, 1].imshow(
            np.vstack(
                2 * [stripes3[i, :, 1]]),  # vstack otherwise imshow complains
            aspect=100,
            cmap=Set3_12.mpl_colormap,
            interpolation="nearest")
        axes[i, 1].xaxis.set_ticks_position('bottom')
        axes[i, 1].yaxis.set_visible(False)
    axes[0, 0].set_title("Infomap")
    axes[0, 1].set_title("SBM")
    plt.suptitle(title)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.xlabel('Residue number')
    plt.show()


In [58]:
def getInfomapPartition(pdbRef):
    """Given a PDB reference, return the list of lists from the database."""
    global db
    inputArgs = {
        "scaling": 4.0,
        "edgelisttype": "residue",
        "hydrogenstatus": "noH",
        "pdbref": pdbRef,
        "database": db
    }
    proteinNetwork = proteinnetworks.network.Network(**inputArgs)
    partitionArgs = {
        "pdbref": pdbRef,
        "edgelistid": ObjectId(proteinNetwork.edgelistid),
        "detectionmethod": "Infomap",
        "N": 100,
        "database": db
    }
    partition = proteinnetworks.partition.Partition(**partitionArgs)
    return partition

In [60]:
SBMfiles = glob.glob("Data/*.sbm")
SBMfile = np.random.choice(SBMfiles)
with open(SBMfile) as flines:
    # The + 1 is to maintain consistency with Infomap's labelling.
#     print(list(line.split(" ") for line in flines))
    SBMdata = [[int(x) + 1 for x in line.strip().split(" ")]
               for line in flines]
    
#     print(SBMdata)
    
    # Get the corresponding Infomap file
    pdbRef = os.path.basename(SBMfile)[:4]
    print(pdbRef)
    try:
        partition = getInfomapPartition(pdbRef)
    except FileNotFoundError:
        print("Infomap bug")
#         continue
    plotStripeDiagramSideBySide(partition.data, SBMdata, pdbRef)


In [61]:
partition.plotPymolStructure()

#### In order to do a proper comparison of the SBM vs Infomap, we need to look at the structure properly 

In [62]:
def plotPymolStructure(pdbRef, SBMData):
    """Plot the community structure overlaid onto the protein using PyMol.

    Generate the sequence of b-factor alterations for each level.
    Each column of the input Array alters a Pymol Object of the form
    $PDBREF_$INDEX where the pdbref is the 4-character identifier, and index the
    column number (starting from zeros)
    """
    pymolCommands = []

    selector = "resi"
    for i, col in enumerate(SBMData):
        numberOfCommunities = len(set(col))
        col = np.asarray(col, dtype=int)  # necessary for np.where()
        pymolCommand = "\n".join([
            "alter {0}_{1} and ({2} {3} ), b={4}".format(
                pdbRef, i, selector,
                " or {} ".format(selector).join(str(x + 1) for x in np.where(col == com)[0]),
                com / numberOfCommunities) for com in set(col)
        ])
        pymolCommands.append(pymolCommand)
    """
    Amalgamate the pymol commands so as to load a pdb file for each level of hierarchy,
    then run the bfactor alterations.
    """
    # Write the pdb file out as a temp file for PyMol FIXME
    pdb = db.extractPDBFile(pdbRef)
    if not pdb:
        pdb = db.fetchPDBFileFromWeb(pdbRef)
    with open("temp.pdb", mode='w') as flines:
        flines.write("\n".join(pdb))

    pymolScript = "\n".join([
        "load temp.pdb, {0}_{1}".format(pdbRef, i)
        for i in range(len(pymolCommands))
    ])
    pymolScript += "\n"
    pymolScript += "\n".join(pymolCommands)
    pymolScript += """
#formatting
bg_color white
hide all
#show sticks
show cartoon
spectrum b, rainbow,  minimum=0, maximum=1
set opaque_background=0
set antialias = on
set line_smooth = 1
set depth_cue = 1
set specular = 1
set surface_quality = 1
set stick_quality = 15
set sphere_quality = 2
set ray_trace_fog = 0.8
set light = (-0.2,0,-1)

set ray_shadows, 0
set surface_mode, 1
set cartoon_side_chain_helper,on
rebuild
    """

    with open("temp.pml", mode='w') as flines:
        flines.write(pymolScript)

    subprocess.run(["pymol", "temp.pml"])
    os.remove("temp.pml")
    os.remove("temp.pdb")

In [70]:
SBMfiles = glob.glob("Data/*.sbm")
SBMfile = np.random.choice(SBMfiles)
with open(SBMfile) as flines:
    # The + 1 is to maintain consistency with Infomap's labelling.
#     print(list(line.split(" ") for line in flines))
    SBMdata = [[int(x) + 1 for x in line.strip().split(" ")]
               for line in flines]
    
    pdbRef = os.path.basename(SBMfile)[:4]
    print(pdbRef)
    try:
        partition = getInfomapPartition(pdbRef)
    except FileNotFoundError:
        print("Infomap bug")
#         continue
    plotPymolStructure(pdbRef, SBMdata)
    partition.plotPymolStructure()

In [ ]:
plotPymolStructure()